# ATOM : an Artificial Trading Open Market

Auteurs : Rémi Morvan & Philippe Mathieu [CRISTAL Lab](http://www.cristal.univ-lille.fr), [SMAC team](https://www.cristal.univ-lille.fr/?rubrique27&eid=17), [Lille University](http://www.univ-lille.fr)

email : philippe.mathieu at univ-lille.fr

Date : 2018

# Principe général
ATOM est un modèle de marché financier dirigé par les ordres dans lequel des agents traders artificiels peuvent intéragir. Il permet notamment de voir les conséquences d'exécution de séries d'ordres, de tester des comportements de traders spécifiques ou de tester des règles de régulation de marché. ATOM s'appuie sur une technologie multi-agents, branche de l'IA étudiant les interactions entre entités artificielles et leurs interactions. ATOM peut gérer des milliers d'agents simultanément sur un marché multi-options à double carnet d'ordres de type Euronext ou NYSE.

Le chargement de la librairie ATOM se fait très classiquement par un import. Il est en général préférable de charger en même temps différentes librairies (notamment numpy, random et statsmodels ainsi que matplotlib pour les tracés graphiques) permettant l'analyse des différentes données produites.

In [ ]:
from atom import *
from data_processing import *
import numpy as np
import random
import statsmodels.tsa.stattools as stats
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15,10)

## Prise en main

Avant toute expérience, il faut créer un marché. Pour créer un marché, il suffit d'instancier la classe `Market(assets)`, où `assets` contient la liste des assets traités par ce marché. Par défaut, il existe deux types d'agents :
* Des `dumb agents`, qui n'envoient aucun ordre hormis ceux qu'on leur demande d'envoyer
* Des `ZIT`, qui envoient, à chaque fois qu'ils ont la parole, un ordre aléatoire, dont le prix est tiré uniformément entre 1000 et 9999 et la quantité entre 1 et 9.

Pour créer un agent, on lui indique la référence du marché en la passant au constructeur. Une fois l'agent créé, on indique au marché le nom de ce nouvel agent à l'aide de la méthode `add_trader`.

### Dumb Agents

L'utilisateur que vous êtes ne peut pas parler directement au marché. Seuls des agents parlent au marché. Un dumb agent est un simple intermédiaire entre nous et le marché afin que l'on puisse envoyer un ordre si on le souhaite. Pour cela, le dump agent possède une méthode `send_order` qui prend en paramètres un ordre et le marché auquel l'envoyer.

L'ordre le plus standard sur les marchés dirigés par les ordre est le limit order, constitué d'un quintuplet, que l'on créé grâce à la classe `LimitOrder(asset, source, direction, price, qty)`.

In [ ]:
m = Market(['Apple', 'Google']) # Créé un marché dont les assets sont 'Apple' et 'Google'
t = DumbAgent(m, name='pauline') # Créé un dumb agent
m.add_trader(t) # Rajoute le dumb agent au marché

t.send_order(LimitOrder('Apple', t, 'ASK', 5000, 10), m)
# Le dumb agent a envoyé au marché un ordre de vente de 10 actions Apple au prix 5000
t.send_order(LimitOrder('Apple', t, 'BID', 6000, 5), m)
# Le dumb agent a envoyé au marché un ordre d'achat de 5 actions Apple au prix 6000

### Trace

Atom peut fournir durant l'exécution une trace de toutes les opérations effectuées. Cela permet de voir précisément quelles sont les opérations réalisées par les agents et les conséquences de ces différentes opérations. Les premières lignes de cette trace (commençant par '#')  précisent la syntaxe utilisée par ATOM.
* Les lignes LimitOrder et CancelMyOrders sont affichées quand un ordre (LimitOrder ou CancelMyOrders) a été envoyé
* Les lignes Tick correspondent à une fin de tick
* Les lignes Price sont affichées quand un prix est fixé (long fixing)
* Les lignes NewAgent sont affichées quand un agent est rajouté au marché
* Les lignes Agent sont affichées quand un agent a son cash ou sa quantité d'actions qui sont modifées
* Les lignes AgentWealth correspondent au wealth des agents

On peut diriger cette trace sur différentes sorties (écran, fichier) ou ne pas activer cette trace, à l'aide du paramètre `out` lors de la création du marché :
* Si rien n'est précisé, la trace est affichée dans la console
* Si `out = None`, rien n'est affiché
* Si `out = FileObject`, la trace est affichée dans un fichier. Si on utilise cette méthode, il est impératif de penser à fermer le fichier après qu'on ait fini d'écrire dedans à l'aide de la méthode `close()`.

In [ ]:
file = open('trace.dat', 'w') # On ouvre le fichier trace.dat en écriture

m = Market(['Apple'], out=file) # On donne ce fichier au marché
t = DumbAgent(m)
m.add_trader(t)
t.send_order(LimitOrder('Apple', t, 'ASK', 5000, 10), m)

file.close() # On n'oublie pas de fermer le fichier.

Une méthode `print_state()` fournie par marché, affiche une synthèse de l'exécution, avec notamment : le nombre d'ordres reçus, le nombre de prix fixés et le nombre d'ordres ASK et BID en attente.

In [ ]:
m = Market(['Apple'])
t = DumbAgent(m, name='paul')
m.add_trader(t)
t.send_order(LimitOrder('Apple', t, 'ASK', 5000, 10), m)
t.send_order(LimitOrder('Apple', t, 'BID', 6000, 5), m)
m.print_state()

### Exercice
Par défaut cet agent ne possède rien. `cash` à zéro et assets à zéro pour tous les titres.
Refaites l'experience avec 10000 de cash et 100 titres Apple


### ZITs

Les ZITs peuvent être créés de la même façon que les dumb agents, en utilisant la classe `ZITTrader(m)`. A la différence des dumb agents, les ZIT traders sont totalement autonomes. Une fois les ZITs créés et ajoutés au marché, on peut utiliser la méthode du marché `run_once()`, qui donne une fois la parole à chaque agent sur chaque asset. c'est le marché qui décide de la séquence de prise de parole, afin de n'avantager personne. Deux exécutions  peuvent donc donner 2 prises de parole différentes. Afin de pouvoir les identifier, les ZIT ont un nom qui porte un numéro.

In [ ]:
m = Market(['Apple'])
m.add_trader(ZITTrader(m))
m.add_trader(ZITTrader(m))
m.run_once()

Afin de faciliter la création d'un grand nombre d'agents, le marché fournit la méthode `generate(nb_ZIT, nb_turn)`, qui créé nb ZITS automatiquement, les ajoute au marché et exécute nb_turn tours de parole. Le code précédent est donc équivalent au code suivant :

In [ ]:
m = Market(['Apple'])
m.generate(2, 1)

### Pour aller plus loin...

Finalement, tous les agents ont deux paramètres facultatifs :
* `initial_assets`, qui est une liste de même taille que la liste des assets du marché, et qui contient l'ensemble des assets dont disposent les agents. Si rien n'est précisé, on suppose que c'est une liste remplie de zéros.
* `cash`, qui est le cash initial dont dispose l'agent et qui vaut 0 si rien n'est précisé.

Par exemple, si `m = Market(['Apple', 'Google'])`, alors la commande `m.add_trader(ZITTrader(m, [5, 10], 5000))` rajoute au marché un ZIT qui dispose initialement d'un cash de 5000, de 5 actions d'Apple et de 10 actions de Google.

La méthode `generate` dispose elle de deux paramètres `init_assets` et `init_cash`, qui sont tous deux des entiers. Chacun des ZIT est alors créé avec un cash initial égal à `init_cash` et disposant, pour chaque asset, de `init_assets` actions.

En outre, le marché possède un paramètre optionnel `fix`, valant `'L'` par défaut (long fixing), et pouvant aussi valoir `'S'` (short fixing).

Finalement, on peut passer au marché un paramètre `trace`, auquel on peut donner les valeurs suivantes :
* `'all'` : toutes les informations sont écrites dans la trace, même les carnets d'ordres
* `'all except orderbooks'` (valeur par défaut) : comme `all`, mais les carnets d'ordres ne sont pas écrits
* une sous-liste de `['order', 'tick', 'price', 'agent', 'newagent', 'wealth', 'orderbook']` : les valeurs présentes dans cette sous-lignes correspondent aux types d'informations qu'on veut voir écrites dans la trace

Par exemple, si `trace=['price', 'wealth']`, alors seules les lignes "Price" et "AgentWealth" seront écrites.

In [ ]:
m = Market(['Apple', 'Google'], trace='all')
m.generate(2, 3, 10, 10000)
m.print_state()
# Créé 2 ZIT, qui vont faire 3 tours de parole
# et qui ont initialement 10000 de cash, 10 actions Apple et 10 actions Google

Les orderbooks sont affichés à chaque fois qu'ils sont modifiés.

## Utilisation de la trace pour afficher différentes courbes

In [ ]:
file = open('trace.dat', 'w')

m = Market(['Apple', 'Google'], out=file)
m.generate(3, 100, 10, 0)
m.print_state()

file.close()

### Affichage des prix

On peut extraire les prix de la trace à l'aide de la fonction `extract_prices`, qui prend en entrée un nom de fichier, et retourne un dictionnaire dont les clés sont les assets et les valeurs sont des tuples (T, P), avec T la liste des timestamps et P la liste des prix correspondants (pour un asset donné).

In [ ]:
Prices = extract_prices('trace.dat')
for asset in Prices.keys():
    plt.plot(Prices[asset][0], Prices[asset][1], '-', label=asset)
plt.legend(loc='best')
plt.xlabel('Time')
plt.ylabel('Price')
plt.show()

### Affichage de l'évolution du wealth des agents

À la fin d'une simulation, on peut accéder au wealth d'un agent avec la méthode `get_wealth` (qui prend le marché en paramètre). Par exemple, on peut afficher les caractéristiques (avec la méthode `get_infos`) de l'agent le plus riche et du plus pauvre:

In [ ]:
t_max = m.traders[0]
t_min = m.traders[0]
for t in m.traders: # On parcourt l'ensemble des traders
    if t.get_wealth(m) > t_max.get_wealth(m):
        t_max = t
    elif t.get_wealth(m) < t_max.get_wealth(m):
        t_min = t
print("Agent le plus riche - "+t_max.get_infos(m))
print("Agent le plus pauvre - "+t_min.get_infos(m))

Il est bien sûr possible d'afficher l'évolution du wealth de tous les agents, en lisant la trace : la fonction `extract_wealths` prend un nom de fichier en entrée et retourne un dictionnaire dont les clés sont les agents et les valeurs sont des listes (T, W) où T est la liste des timestamps et W la liste des wealths de cet agent.

In [ ]:
Wealth = extract_wealths('trace.dat')
for agent in Wealth.keys():
    T, W = Wealth[agent]
    plt.plot(T, W, '-', label=agent)
plt.legend(loc='best')
plt.grid()
plt.xlabel('Tick')
plt.ylabel('Wealth')
plt.show()

### Affichage des rentabilités

Grâce à la librairie numpy il devient facile d'afficher la séquence des rentabilités (géométrique ou logarithmique): si $p_n$ est le $n$-ème élément de Prices, alors
> (Prices[1:]-Prices[:-1])/Prices[:-1] (resp. np.log(Prices[1:]) - np.log(Prices[:-1]))

donne la séquence des $\dfrac{p_{n+1}-p_n}{p_n}$ (resp. $\ln(p_{n+1})-\ln(p_n)$).

In [ ]:
asset = 'Apple'
Prices = np.array(extract_prices('trace.dat')[asset][1])
Returns = (Prices[1:]-Prices[:-1])/Prices[:-1]
Returns_eco = np.log(Prices[1:]) - np.log(Prices[:-1])
plt.plot(Returns, '-', label="Returns (growth rate)")
plt.plot(Returns_eco, '-', label="Returns (log difference)")
plt.axhline(0, color='k')
plt.ylabel('Return ('+asset+')')
plt.legend(loc='best')
plt.show()

On peut aussi afficher l'histogramme des rentabilités (définies comme la différence logarithmique). Pour cela, on dispose d'une fonction `draw_returns_hist` qui prend en entrées le nom du fichier dans lequel la trace est stockée, le nom de l'asset pour lequel on va calculer les rentabilités, et un nombre de points. En retour, on obtient un tuple (R, D, N) où R est une liste des rentabilités, D la liste de leur densités et N la densité de la loi normale de même espérance et de même écart-type. La fonction trace cet histogramme comparé à la gaussienne de même espérance et même écart type.

In [ ]:
#Attention : ce bloc met de l'ordre de 10sec à s'exécuter

# On utilise ici generate avec 10 agents durant 10.000 tours (donc 100.000 ordres envoyés au marché !)
# Mieux vaut avoir beaucoup de points, même si c'est un peu long à calculer.
file = open('trace.dat', 'w')
asset = 'Apple'
m = Market([asset], out=file, trace=['price'])
# On n'écrit que les prix dans la trace : ce sont les seules lignes dont on a besoin
# et cela permet de gagner un peu de temps de calcul
m.generate(10, 10000)
file.close()

In [ ]:
draw_returns_hist('trace.dat', asset, 100)

On observe bien le fait stylisé associé à la rentabilité : par rapport à une gaussienne de même espérance et même écart type, on a une forte kurtosis (pic central de plus grande amplitude) et des queues plus épaisses.

Remarque : Pourquoi choisir la différence logarithmique plutôt que le taux d'accroissement ?
Parce que la différence logarithmique a une belle propriété : si on passe d'un prix $p$ à un prix $p'$, alors la rentabilité sera de $r_1 = \log(p') - \log(p)$ ; si on passe de $p'$ à $p$, on a une rentabilité de $r_2 = log(p) - log(p') = -r_1$. Cette propriété n'est pas vérifiée par le taux d'accroissement : si on se limite à des prix tirés entre 1000 et 10000, le taux d'accroissement maximal est 9, et le taux d'accroissement minimal est $-0.9$. La distribution des rentabilités définies comme taux d'accroissement ne sera donc pas symétrique.

Remarque 2 : Cette distinction est importante car on travaille avec ZIT qui font que les prix peuvent subir de fortes variations soudaines. Sur un vrai marché, ces variations sont faibles, et donc si on passe d'un prix $p$ à un prix $p'$, on a $\Delta(p) = |p'-p| << p$, donc : $log(p') - log(p) = \log\left(1+\dfrac{p'-p}{p}\right) \sim \dfrac{p'-p}{p}$. Ainsi, sur un vrai marché, peut importe la définition choisie, on aura grosso modo le même résultat.

### Histogramme des rentabilité quand les prix sont fixés aléatoirement

Certains pensent parfois que les marchés suivent une marche aléatoire. Ce petit exemple montre que le fait stylisé que l'on obtient avec un marché financier à carnet d'ordres asynchrone comme ATOM ne peut être obtenu simplement avec des prix fixés aléatoirement.
Pour illustrer cela, nous allons générer une fausse trace dans laquelle des prix (500000) sont fixés aléatoirement uniformément entre 1 et 100.

In [ ]:
# On créé notre ausse trace dans laquelle les prix sont fixés aléatoirement.
out = open('fake_trace.dat', 'w')
t0 = int(time.time()*1000000)
for i in range(5000000):
    out.write("Price;Apple;Agent 0;Agent 0;%i;1;%i\n" % (random.randint(1000, 9999), int(time.time()*1000000)-t0))
out.close()

De la même manière que précédemment, on affiche l'histogramme des rentabilités.

In [ ]:
draw_returns_hist('fake_trace.dat', 'Apple', 100)

On constate que l'on n'observe plus une sorte de gausienne avec un gros pic et des queues épaisses, mais, près du centre, deux demi-droites qui aux extrémités ont une queue moins épaisse que la gaussienne. Les prix fixés par un système à carnet d'ordres ne sont donc clairement pas de même structure qu'un simple tirage aléatoire, même si les agents eux mêmes ne font que des choix aléatoire.

### Affichage de l'autocorrélation des rentabilités

In [ ]:
Prices = np.array(extract_prices('trace.dat')[asset][1])
Returns = np.log(Prices[1:]) - np.log(Prices[:-1])
acf = stats.acf(Returns, nlags=20)
plt.plot(range(21), acf, 'o', color="orange")
plt.bar([x+0.02 for x in range(21)], acf, .04, color="orange")
plt.axhline(0, color='k')
sigma = max(np.abs(acf[10:]))
plt.axhline(sigma, color='k', linestyle='--')
plt.axhline(-sigma, color='k', linestyle='--')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.show()

## Utilisation de replay

ATOM peut aussi être utilisé en "flow-replayer". La méthode `replay` du marché permet de inscrites dans ce fichier. Le contenu de ce type de fichier est exactement le même qu'un fichier trace de ATOM, mais Replay ne lit que les créations d'agents et les ordres envoyés (les lignes "NewAgent" et "Order").
Il est donc possible de générer une trace avec `generate`et rejouer le fichier obtenu via `replay`. ATOM offre comme on lpeut le voir, un cercle vertueux.

In [ ]:
m = Market(['LVMH'], trace=['price'])
m.replay('orderFileEx1.dat')
m.print_state()

On va regarder ce qu'il se passe si on choisit le short fixing.

In [ ]:
m = Market(['LVMH'], trace=['price'], fix='S')
m.replay('orderFileEx1.dat')
m.print_state()

Avec le short fixing, on constate que 7 prix qui sont fixés, contre 9 pour le long fixing.

### Cercle vertueux

On va générer une trace avec `generate`, puis faire un `replay` sur cette trace. On devrait observer que l'état final du système est identique dans les deux cas. Pour être sûr que `replay` ne triche pas, on va uniquement afficher les ligne NewAgent et LimitOrder dans la trace.

In [ ]:
file = open('trace.dat', 'w')
m = Market(['Apple', 'Google', 'Microsoft'], out=file, trace=['newagent', 'order'])
m.generate(2, 1000, init_assets=10, init_cash=100000)
file.close()

for t in m.traders:
    print(t.get_infos(m))

In [ ]:
m2 = Market(['Apple', 'Google', 'Microsoft'], out=None)
m2.replay('trace.dat')

for t in m2.traders:
    print(t.get_infos(m2))

C'est bien ce qu'on observe ! (Nos agents ont juste des noms différents.)